In [1]:
# Magic comands
%reload_ext autoreload
%autoreload 2

# Libraries
import re
import numpy as np
import pandas as pd
import os, sys, webcolors
import plotly.express as px
from datetime import datetime
import plotly.graph_objects as go

module_path = os.path.abspath(os.path.join('.'))
if module_path not in sys.path:
    sys.path.append(module_path)

origin_path = os.getcwd()
parent_dir = os.path.abspath(os.path.join(origin_path, os.pardir))
os.chdir(parent_dir)

from app.nqueens import NQueensGeneticAlgorithm, NQueensBacktracking, NQueensHillClimbing, NQueensSimulatedAnnealing

## __1. Algorithm Functions__

### __1.1 Backtracking__

In [2]:
def backtracking_algorithm(n= 8):
    start_time = datetime.now()
    backtracking = NQueensBacktracking()
    results = backtracking.solveNQueens(n)
    end_time = datetime.now()
    execution_time = int((end_time - start_time).total_seconds() * 1000)

    return {'dimensions': n, 'duration': execution_time}

In [3]:
# testing
ba = backtracking_algorithm(n = 12)

ba

{'dimensions': 12, 'duration': 1003}

### __1.2 Hill Climbing__

In [4]:
def hill_climbing(n=8):
    nQueensHC = NQueensHillClimbing(dimension=n)
    nQueensHC.run()
    return nQueensHC.report()


In [5]:
# testing
hc = hill_climbing(n = 12)

hc

{'duration': 78,
 'best_fitness': 64,
 'best_fitness_percentage': 96.96969696969697,
 'best_representation': [4, 0, 7, 5, 1, 11, 9, 8, 2, 2, 10, 6]}

### __1.3 Simulated Annealing__

In [6]:
def sim_annealing(n=8):
    nQueensSA = NQueensSimulatedAnnealing(dimension=n)
    nQueensSA.run()
    return nQueensSA.report()


In [7]:
# testing
sa = sim_annealing(n = 12)

sa

{'duration': 1529,
 'best_fitness': 65,
 'best_fitness_percentage': 98.48484848484848,
 'best_representation': [3, 5, 8, 2, 4, 10, 1, 7, 9, 11, 6, 0]}

### __1.4 Genetic Algorithm__

In [8]:
def genetic_algorithm(
                    n = 12,
                    pop = 100,
                    gen = 50,
                    mut_perc = .05, # .01 .1
                    cross_perc = .75, # .6 to .9
                    mut='random_mutation',
                    cross='single_cross',
                    select='tournament_selection',
                    ):
    
    nQueensGA = NQueensGeneticAlgorithm(population_size=pop, dimension=n)

    nQueensGA.run(
        generations=gen,
        mutation_probability=mut_perc,
        crossover_probability=cross_perc,
        mutation_operator=mut,
        crossover_operator=cross,
        selection_operator=select
    )

    return nQueensGA.report()

In [9]:
# testing function
ga = genetic_algorithm(
                                            n = 8,
                                            pop = 200,
                                            gen = 100,
                                            mut_perc = .05,
                                            cross_perc = .75,
                                            mut = "swap_mutation",
                                            cross = "pmx",
                                            select = "fps"
                                            )
ga

{'generations': 100,
 'duration': 408,
 'best_fitness': 28,
 'best_fitness_percentage': 100.0,
 'best_representation': [1, 4, 0, 6, 5, 2, 7, 3],
 'worst_fitness': 28,
 'worst_representation': [1, 4, 0, 6, 5, 2, 7, 3],
 'mean_fitness': 28.0,
 'selection_operator': 'fps',
 'mutate_operator': 'swap_mutation',
 'crossover_operator': 'pmx'}

## __1. Operators & Paramaters Optimization__

In [10]:
# dictionary with parameter names and default values 
defaults = genetic_algorithm.__defaults__
parameters = genetic_algorithm.__code__.co_varnames
params = {param: val for param, val in zip(parameters, defaults)}

# color list for plots
color_palette = ['gray', 'steelblue', 'purple', 'darkgray', 'lightgray', 'slategray', 'cornflowerblue', 'lavender', 'powderblue']

In [11]:
def best_operators_search(parameter: str, values: list, previous_best_params: list = None):
    
    if previous_best_params:
        params = previous_best_params
    else:
        # dictionary with parameter names and default values 
        defaults = genetic_algorithm.__defaults__
        parameters = genetic_algorithm.__code__.co_varnames
        params = {param: val for param, val in zip(parameters, defaults)}
    
    # initializing variables
    best_fit = 0.0
    time_fit = 9e99
    fit_dict = {}
    time_dict = {}
    param_list = []

    # Perform grid search
    for i in values:
        # 100 iterations to generate statistically relevant results
        fit_list = []
        time_list = []
        for _ in range(100):
            # changing parameter according to input
            params[parameter] = i
            # Define the model with current hyperparameters
            ga = genetic_algorithm(**params)
            # gets restults
            fit = ga['best_fitness_percentage']
            time_it = ga['duration']
            # Check if current solution has better fitness than previous best 
            if fit > best_fit:
                best_fit = fit
                time_fit = time_it
                best_param = {parameter: i}
            # Check if the solution with fitness as good is faster than previous best
            if fit == best_fit and time_it < time_fit:
                time_fit = time_it
                best_param = {parameter: i}
            # Append results to get 100 iterations values
            fit_list.append(fit)
            time_list.append(time_it)
        # saving results per parameter
        param_list.append(best_param)
        fit_dict['Fit_' + str(i)] = fit_list
        time_dict['Time_' + str(i)] = time_list

    unique_best_params =list({tuple(d.items()): d for d in param_list}.values())

    # dataframe with results
    fit_dict.update(time_dict)
    df = pd.DataFrame(fit_dict)
        
    return unique_best_params, df

In [12]:
def plot_values(data: pd.DataFrame, colors: list, layout: int, template:str, grid:  bool):
    # Filter columns starting with "Fit_"
    fit_columns = [col for col in data.columns if col.startswith('Fit_')]

    # Filter columns starting with "Time_"
    time_columns = [col for col in data.columns if col.startswith('Time_')]

    template_map = {'white':'plotly_white', 'dark': 'plotly_dark'}

    tick_font_map = {'white': webcolors.rgb_to_hex(webcolors.name_to_rgb('black')), 'dark': webcolors.rgb_to_hex(webcolors.name_to_rgb('white'))}

    # Layout styles
    layout_1 = {
        'xaxis_title': 'Iterations',
        'yaxis_title': 'Time (ms)',
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'title': {'text': 'Time Comparison', 'font': {'size': 19, 'family': 'Verdana'}},
        'xaxis': {'showgrid': grid, 'ticks': 'outside', 'tickfont': {'color': tick_font_map[template]}},
        'yaxis': {'showgrid': grid, 'tickfont': {'color': tick_font_map[template]}}
        }
    
    layout_2 = {
        'template': template_map[template],
        'font': {'size': 14, 'family': 'Verdana'},
        'xaxis': {
            'showline': True,
            'showgrid': grid,
            'showticklabels': True,
            'linewidth': 1.5,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}},
        'yaxis': {
            'showgrid': grid,
            'zeroline': False,
            'showline': False,
            'showticklabels': True,
            'ticks': 'outside',
            'tickfont': {
                'family': 'Verdana',
                'size': 11,
                'color': tick_font_map[template]}}}
    
    layout_map = {1:layout_1, 2:layout_2}

    # Convert color names to RGB values
    rgb_colors = [webcolors.name_to_rgb(color) for color in colors]

    mode_size = 8
    line_size = 2

    # Create a subplot with two line plots
    fig = go.Figure()

    # Add line plots for "Fit_" columns
    i = 0
    for col in fit_columns:
        fig.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Fit_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    layout_map[layout].update({
                            'title': 'Fitness Comparison',
                            'xaxis_title': 'Iterations',
                            'yaxis_title': 'Fitness'})

    fig.update_layout(layout_map[layout])

    fig2 = go.Figure()

    # Add line plots for "Time_" columns
    i = 0
    for col in time_columns:
        fig2.add_trace(go.Scatter(x=data.index, y=data[col], name=col.replace('Time_', ''),
                                line=dict(color=colors[i], width=line_size),
                                ))

        fig2.add_trace(go.Scatter(
            x=[data.index[0], data.index[-1]],
            y=[data[col][0], data[col].iloc[-1]],
            mode='markers',
            marker=dict(color=colors[i], size=mode_size),
            name=''
        ))
        i += 1

    layout_map[layout].update({
                            'title': 'Time Comparison',
                            'xaxis_title': 'Iterations',
                            'yaxis_title': 'Time (ms)'})

    fig2.update_layout(layout_map[layout])

    # Show the plots
    fig.show()
    fig2.show()

### __2.1 Mutation Operator__

In [13]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
mutation = ['swap_mutation', 'random_mutation', 'inversion_mutation']

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('mut', mutation, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Setting the best parameter as new default value
replace_param = {'mut': best_param_time}

# Plotting results
plot_values(data = df, colors = color_palette[:len(mutation)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 12, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'random_mutation', 'cross': 'single_cross', 'select': 'tournament_selection'}

Average results:


,Fit_swap_mutation,Fit_random_mutation,Fit_inversion_mutation,Time_swap_mutation,Time_random_mutation,Time_inversion_mutation
0,96.227,96.364,96.348,9.37,10.05,9.41



Best Fitness: random_mutation

Best Time: swap_mutation



### __2.2 Crossover Operator__

In [14]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
xo = ['single_cross', 'cycle_cross', 'pmx', 'arithmetic_cross']

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('cross', xo, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'cross': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(xo)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 12, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'swap_mutation', 'cross': 'single_cross', 'select': 'tournament_selection'}

Average results:


,Fit_single_cross,Fit_cycle_cross,Fit_pmx,Fit_arithmetic_cross,Time_single_cross,Time_cycle_cross,Time_pmx,Time_arithmetic_cross
0,96.242,96.424,96.121,96.288,9.38,11.37,11.89,14.18



Best Fitness: cycle_cross

Best Time: single_cross



### __2.3 Selection Operator__

In [15]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
select = ['tournament_selection', 'fps', 'rank']

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('select', select, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'select': best_param_fit}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(select)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 12, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'swap_mutation', 'cross': 'cycle_cross', 'select': 'tournament_selection'}

Average results:


,Fit_tournament_selection,Fit_fps,Fit_rank,Time_tournament_selection,Time_fps,Time_rank
0,96.288,96.409,96.409,11.43,56.17,65.23



Best Fitness: fps

Best Time: tournament_selection



### __2.4 Population__

In [16]:
# seed for reproducibility
np.random.seed(0)

# Define the range of values
population = [100, 150, 200]

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('pop', population, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'pop': int(best_param_fit)}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(population)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 12, 'pop': 100, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'swap_mutation', 'cross': 'cycle_cross', 'select': 'fps'}

Average results:


,Fit_100,Fit_150,Fit_200,Time_100,Time_150,Time_200
0,95.955,96.333,96.879,57.45,121.97,202.88



Best Fitness: 200

Best Time: 100



### __2.5 Generations__

In [17]:
# seed for reproducibility
np.random.seed(0)

generations = [50, 100, 150]

# Setting previous best parameter as new default value
params.update(replace_param)

# Print current parameters to use in best_operators_search
print('\033[1mParameters for GA:\033[0m', end='\n\n')
print(params, end='\n\n')

param_list, df = best_operators_search('gen', generations, params)

# Displaying average results of the 100 iterations
print('\033[1mAverage results:', end='\n')
df['const'] = 'const'
df_avg = df.groupby('const').mean().reset_index(drop=True).round(3)
display(df_avg)

# Getting best parameter by fitness & time
best_param_fit = df_avg[[col for col in df_avg.columns if col.startswith('Fit_')]].idxmax(axis=1)[0].replace('Fit_', '')
best_param_time = df_avg[[col for col in df_avg.columns if col.startswith('Time_')]].idxmin(axis=1)[0].replace('Time_', '')
replace_param = {'gen': int(best_param_fit)}
print(f'\nBest Fitness: \033[1m{best_param_fit}\033[0m\n\nBest Time: \033[1m{best_param_time}\033[0m', end='\n\n')

# Plotting results
plot_values(data = df, colors = color_palette[:len(generations)], layout=2, template='white', grid=True)

Parameters for GA:

{'n': 12, 'pop': 200, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'swap_mutation', 'cross': 'cycle_cross', 'select': 'fps'}

Average results:


,Fit_50,Fit_100,Fit_150,Time_50,Time_100,Time_150
0,96.848,96.682,96.848,205.54,415.32,620.16



Best Fitness: 50

Best Time: 50



In [18]:
# Setting previous best parameter as new default value
params.update(replace_param)

print('\033[1mFinal Parameters for GA:\033[0m', end='\n\n')
print(params)

Final Parameters for GA:

{'n': 12, 'pop': 200, 'gen': 50, 'mut_perc': 0.05, 'cross_perc': 0.75, 'mut': 'swap_mutation', 'cross': 'cycle_cross', 'select': 'fps'}


## __3. Algortihms Comparison__

In [59]:
dimensions = [5, 8]
fit = {}
time = {}

algorithm_idx_map = {
                    1: 'Bactracking',
                    2: 'Hill Climbing',
                    3: 'Simulated Annealing',
                    4: 'Genetic Algorithm'
                    }

for n in dimensions:
    ba_fit_list = []
    hc_fit_list = []
    sa_fit_list = []
    ga_fit_list = []

    ba_time_list = []
    hc_time_list = []
    sa_time_list = []
    ga_time_list = []

    for _ in range(100):
        ba = backtracking_algorithm(n = n)
        hc = hill_climbing(n = n)
        sa = sim_annealing(n = n)
        params['n'] = n
        ga = genetic_algorithm(**params)

        ba_fit_list.append(100.)
        hc_fit_list.append(hc['best_fitness_percentage'])
        sa_fit_list.append(sa['best_fitness_percentage'])
        ga_fit_list.append(ga['best_fitness_percentage'])
        ba_time_list.append(ba['duration'])
        hc_time_list.append(hc['duration'])
        sa_time_list.append(sa['duration'])
        ga_time_list.append(ga['duration'])

    fit_list = [ba_fit_list, hc_fit_list, sa_fit_list, ga_fit_list]
    idx = 1
    for ls in fit_list:
        fit[algorithm_idx_map[idx] + ' ' + str(n) + 'Dim Fit'] = ls
        idx += 1

    time_list = [ba_time_list, hc_time_list, sa_time_list, ga_time_list]
    idx = 1
    for ls in time_list:
        time[algorithm_idx_map[idx] + ' ' + str(n) + 'Dim Time'] = ls
        idx += 1

fit.update(time)
df = pd.DataFrame(fit)

fit_cols = [col for col in df.columns if 'Fit' in col]
time_cols = [col for col in df.columns if 'Time' in col]

for n in dimensions:
    ndim_fit_cols = [col for col in fit_cols if str(n) in col]
    ndim_time_cols = [col for col in time_cols if str(n) in col]

    df_melted = pd.melt(df[ndim_fit_cols], var_name='Algorithm', value_name='Value')
    fig = px.box(df_melted, x='Algorithm', y='Value', color='Algorithm')
    fig.update_layout(title=f'{n}-Queens Problem - Fitness per Algorithm',
                    xaxis_title='Algorithm',
                    yaxis_title='Fitness')
    fig.show()

    df_melted = pd.melt(df[ndim_time_cols], var_name='Algorithm', value_name='Value')
    fig = px.box(df_melted, x='Algorithm', y='Value', color='Algorithm')
    fig.update_layout(title=f'{n}-Queens Problem - Time per Algorithm',
                    xaxis_title='Algorithm',
                    yaxis_title='Time (ms)')
    fig.show()